In [1]:
#@param {type:"boolean"}

In [2]:
# gitURL = 'https://gitlab.aiacademy.tw/junew/federated_aia_test.git'
# account = 'at102091:12345678'

with open('./account.cfg', 'r') as f:
    r = f.read()
    
    dic = {}
    for i in r.splitlines():
        i = [item.strip() for item in i.split('=')]
#         print(i.split('='))
        dic[i[0]] = i[1]
    print(dic)
gitURL = dic['gitURL']
account = dic['account']
repo_name = 'june-federated-server'

{'gitURL': 'https://gitlab.com/aia-federated-0121/june-federated-server.git', 'account': 'aiafederated0121:federated0121'}


In [3]:
import os
if os.getcwd().split('/')[-2:] == ['FL_June', 'server']:
    os.popen('git clone https://{account}@{gitURL}'.format(account=account,
                                                           gitURL=gitURL.split('//')[-1])).read()
    print('clone to ',os.getcwd())
else:
    print(os.getcwd())

clone to  /home/jovyan/git/FL_June/server


fatal: destination path 'june-federated-server' already exists and is not an empty directory.


In [4]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Input, Dense, Dropout, Activation,
                                     BatchNormalization, Flatten,
                                     Conv2D, MaxPooling2D)
import tensorflow as tf
import numpy as np
from datetime import date
import sys
sys.path.append(f'./{repo_name}/')
from utils import compressed_cpickle, decompress_cpickle

2022-01-18 11:18:17.676301: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


### control_key (optional)

In [5]:
contro_key = {}
contro_key['new_model'] = False # default to False


## 移動到federated_aia_test floder
如果不存在，請先執行最上面的git clone

In [6]:
print(os.getcwd())
if os.getcwd().split('/')[-2:] == ['server', 'june-federated-servert']:
    pass
else:
    os.chdir('../../FL_June/server/june-federated-server')
    os.getcwd()

/home/jovyan/git/FL_June/server


## 建立新的初始化global model 
> 只有當模型不存在、或者你更新了架構、打算重新訓練的時候

In [7]:
def simplecnn():
    # 選擇 Keras 的 API 寫法
    inputs = Input(shape=(28,28,1))
#     inputs = inputs
    # 第一層
    # 建立卷積層，設定32個3*3的filters
    # 設定ReLU為激活函數。
    x = Conv2D(32, (3, 3), activation='relu')(inputs)

    # 第二層 - 卷積層 + 池化層
    x = Conv2D(32, (3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # 第三層 - 卷積層
    x = Conv2D(64, (3, 3), activation='relu')(x)

    # 第四層 - 卷積層 + 池化層
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    # 建立分類模型 (MLP) : 平坦層 + 輸出層 (10)
    x = Flatten()(x)
    outputs = Dense(10, activation='softmax')(x)

    cnn_model = Model(inputs=inputs, outputs=outputs)
    return cnn_model

In [8]:
from datetime import date
import pathlib

today = date.today()
print(today)

path = '../saved_model'
pathlib.Path(f'{path}').mkdir(parents=True, exist_ok=True)     
lis = os.listdir(path)
lis = [i for i in lis if i.__contains__('global_model')]
print(lis)


2022-01-18
[]


In [9]:
if contro_key['new_model'] == True:
    import os
    import shutil
    from datetime import date
    
    # 保留舊模型到server本機 (saved_model被登錄在.gitignore)
    os.makedirs('../saved_model', exist_ok=True)
    if os.listdir().__contains__('global_model.pbz2'):
        today = date.today()
        lis = os.listdir('../saved_model')
        lis = [i for i in lis if i.__contains__('global_model')]
        
        new_model_name = 'global_model_{ver}_{today}.pbz2'.format(today=today, ver=len(lis))
        shutil.move('./global_model.pbz2', '../saved_model/'+new_model_name)
        print('Move global model to FL_June/server/saved_model')
    else:
        pass
    model = simplecnn()

    # weight、架構 (json) to pickle

    model_attri = {'weights':model.get_weights(), 'json':model.to_json()}
    
    print('create new model')
    compressed_cpickle('./global_model', model_attri)

    print('update gitrepo global_model.pbz2')
    run_cmd = lambda cmd_lis:[os.popen(i).read() for i in cmd_lis.split('\n')]
    cmd_lis = '''git add .
    git commit -m'global model complete aggregate and update to g'
    git push https://{account}@{gitURL}
    '''.format(account=account, gitURL=gitURL.split('//')[-1])
    
    
    run_cmd(cmd_lis)


## 下載各個branch中的模型壓縮檔

In [10]:
r = os.popen('git pull').read()
print(r)

os.popen('git remote update origin --prune')
lis = os.popen('git branch -r').read().split('\n')[:-1]


all_client_branch = [i for i in  lis if not i.__contains__('main')]

print(all_client_branch)

# 判斷各個分支是否有更新
if len(all_client_branch) <= 0:
    raise ValueError('June: No clients appear')

Already up to date.

['  origin/BxipCjg_WfS0amO30vC6mQ', '  origin/MKkWS7qXBbnugNAEXN8PEg', '  origin/Ntyfcj14jPRiW4iWiNa2PA', '  origin/Rj1cKm2wIS3f3NmKrQgX1g', '  origin/VA6GWv7MfvF1OnnLj0Ra3w', '  origin/ZmUvKOfUgOhilDwuPALTWA']


From https://gitlab.com/aia-federated-0121/june-federated-server
 * [new branch]      Rj1cKm2wIS3f3NmKrQgX1g -> origin/Rj1cKm2wIS3f3NmKrQgX1g


In [11]:
!git branch -a

* main
  remotes/origin/BxipCjg_WfS0amO30vC6mQ
  remotes/origin/HEAD -> origin/main
  remotes/origin/MKkWS7qXBbnugNAEXN8PEg
  remotes/origin/Ntyfcj14jPRiW4iWiNa2PA
  remotes/origin/Rj1cKm2wIS3f3NmKrQgX1g
  remotes/origin/VA6GWv7MfvF1OnnLj0Ra3w
  remotes/origin/ZmUvKOfUgOhilDwuPALTWA
  remotes/origin/main


In [18]:
# all_lis = [i.split('/')[-1] for i in all_client_branch]
# for i in all_lis:
#     if not i in [_.split('.')[0] for _ in lis]:
#         print(i)
#         remote_name = 'origin'
#         branch_name = i
#         os.popen(f'git branch -d {branch_name}')
#         os.popen(f'git push {remote_name} --delete {branch_name}').read()

In [17]:
run_cmd = lambda cmd_lis:[os.popen(i).read() for i in cmd_lis.split('\n')]

for i in all_client_branch:
    origin_branch_name = i.lstrip()
    filename = origin_branch_name.split('/')[-1]+'.pbz2'
    
    print(origin_branch_name, filename)
    
    
    cmd_lis = '''git checkout remotes/{branch_name} {model_attri_pbz2}
    '''.format(branch_name = origin_branch_name, model_attri_pbz2=filename)
    result = os.popen(cmd_lis).read()
    print(result)

origin/BxipCjg_WfS0amO30vC6mQ BxipCjg_WfS0amO30vC6mQ.pbz2

origin/MKkWS7qXBbnugNAEXN8PEg MKkWS7qXBbnugNAEXN8PEg.pbz2

origin/Ntyfcj14jPRiW4iWiNa2PA Ntyfcj14jPRiW4iWiNa2PA.pbz2

origin/Rj1cKm2wIS3f3NmKrQgX1g Rj1cKm2wIS3f3NmKrQgX1g.pbz2

origin/VA6GWv7MfvF1OnnLj0Ra3w VA6GWv7MfvF1OnnLj0Ra3w.pbz2

origin/ZmUvKOfUgOhilDwuPALTWA ZmUvKOfUgOhilDwuPALTWA.pbz2



error: pathspec 'remotes/origin/BxipCjg_WfS0amO30vC6mQ' did not match any file(s) known to git.
error: pathspec 'BxipCjg_WfS0amO30vC6mQ.pbz2' did not match any file(s) known to git.
error: pathspec 'remotes/origin/MKkWS7qXBbnugNAEXN8PEg' did not match any file(s) known to git.
error: pathspec 'MKkWS7qXBbnugNAEXN8PEg.pbz2' did not match any file(s) known to git.
error: pathspec 'remotes/origin/Ntyfcj14jPRiW4iWiNa2PA' did not match any file(s) known to git.
error: pathspec 'Ntyfcj14jPRiW4iWiNa2PA.pbz2' did not match any file(s) known to git.
error: pathspec 'remotes/origin/Rj1cKm2wIS3f3NmKrQgX1g' did not match any file(s) known to git.
error: pathspec 'Rj1cKm2wIS3f3NmKrQgX1g.pbz2' did not match any file(s) known to git.
error: pathspec 'remotes/origin/VA6GWv7MfvF1OnnLj0Ra3w' did not match any file(s) known to git.
error: pathspec 'VA6GWv7MfvF1OnnLj0Ra3w.pbz2' did not match any file(s) known to git.
error: pathspec 'remotes/origin/ZmUvKOfUgOhilDwuPALTWA' did not match any file(s) known to

## 聚合並更新global model

In [14]:
model_attri = decompress_cpickle('./global_model.pbz2')
global_model = tf.keras.models.model_from_json(model_attri['json'])

lis = [i for i in os.listdir() if i.__contains__('pbz2')]
lis.remove('global_model.pbz2')
print(lis)

[]


2022-01-18 11:18:32.782593: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-18 11:18:32.783440: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-01-18 11:18:32.832880: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:06:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2022-01-18 11:18:32.835753: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:07:00.0 name: NVIDIA GeForce GTX 1080 Ti computeCapability: 6.1
coreClock: 1.582GHz coreCount: 28 deviceMemorySize: 10.92GiB deviceMemoryBandwidth: 451.17GiB/s
2022-01-18 11:18:32.838739: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 2 with properties: 
pciBusID: 0000:0f:0

In [15]:
lis = [i for i in os.listdir() if i.__contains__('pbz2')]
lis.remove('global_model.pbz2')

weights = []
for i in lis:
    model_attri = decompress_cpickle(i)
    weights.append(model_attri['weights'])
print(np.shape(weights))

(0,)


In [16]:
new_weights = list()
if len(weights) == 0:
    print('no new client to aggregate')
    pass
elif len(weights) == 1:
    print('only single participant')
    new_weights = weights[0]
else:
    for i in zip(*weights):
        new_weights.append(tf.reduce_sum(i, axis=0))
        
global_model.set_weights(new_weights)

no new client to aggregate


ValueError: You called `set_weights(weights)` on layer "model" with a weight list of length 0, but the layer was expecting 10 weights. Provided weights: []...

In [ ]:
model_attri = {'weights':global_model.get_weights(), 'json':global_model.to_json()}

compressed_cpickle('./global_model', model_attri)

In [ ]:
import os

run_cmd = lambda cmd_lis:[os.popen(i).read() for i in cmd_lis.split('\n')]

cmd_lis = '''mv ./global_model.pbz2 ../
rm *.pbz2
mv ../global_model.pbz2 ./
git add .
git commit -m'global model complete aggregate and update to Gmodel'
git push https://{account}@{gitURL}
'''.format(account=account, gitURL=gitURL.split('//')[-1])


run_cmd(cmd_lis)
branch = os.popen('git branch -a').read()
print(branch)